In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np
import pymongo

In [7]:
# Read CSV into dataframe
brew_csv = "../data/beer_reviews.csv"
brewer_df = pd.read_csv(brew_csv)

craft_csv = "../data/craft_beer_fest_data3.csv"
craft_df = pd.read_csv(craft_csv)

craft_df.rename(columns={'Beer':'beer_name'}, inplace=True)

brewer_df.head()
# craft_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [8]:
# Sort on breed and get average times at shelter 
group_by_beer_name_mean = brewer_df.groupby('beer_name').mean()
group_by_beer_name_mean = group_by_beer_name_mean.reset_index()
# group_by_beer_name_mean.head(20)

al_beer_df = pd.merge(group_by_beer_name_mean, craft_df, on=["beer_name"], how='outer')
al_beer_df = al_beer_df.dropna(how='any')
al_beer_df

# test = beer_names[1]
# test = test.replace('"', '')
# test
# name_df = group_by_beer_name_mean["beer_name"]
# name_df.head(20)

# time_in_shelter_days = group_by_breed['time_in_shelter_days'].mean()
# outcome_age_days = group_by_breed['outcome_age_days'].mean()
# intake_age_days = group_by_breed['intake_age_days'].mean()

# # Make dataframes for the average calculations
# time_in_df = pd.DataFrame(time_in_shelter_days)
# outcome_age_df = pd.DataFrame(outcome_age_days)
# intake_age_df = pd.DataFrame(intake_age_days)

# # Concatenate all dataframe averages into single dataframe
# shelter_details_df=pd.concat([time_in_df,outcome_age_df,intake_age_df], axis=1)
# shelter_details_df.head()

,beer_name,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,Brewery,Location,Table,Score,rAvg,Ratings,Bro Score,Style,ABV,URL
59,#9,96.000000,1.215286e+09,3.539528,3.434466,3.501040,3.398058,3.396671,5.100000,299.000000,Magic Hat Brewing Company,"South Burlington, VT",35.0,78.0,3.41,3204.0,74.0,Fruit / Vegetable Beer,5.10%,http://beeradvocate.com/beer/profile/96/299
1820,Allagash Interlude,4.000000,1.233813e+09,4.147790,4.237569,4.074586,4.145028,4.254144,9.500000,27837.000000,Allagash Brewing Company,"Portland, ME",19.0,94.0,4.22,664.0,94.0,Saison / Farmhouse Ale,9.50%,http://beeradvocate.com/beer/profile/4/27837
1844,Allagash White,4.000000,1.199797e+09,4.231954,4.020246,4.029049,3.992958,4.128521,5.200000,59.000000,Allagash Brewing Company,"Portland, ME",19.0,93.0,4.17,2477.0,100.0,Witbier,5.00%,http://beeradvocate.com/beer/profile/4/59
2183,Amber,1923.294231,1.202578e+09,3.577885,3.260577,3.530769,3.373077,3.403846,4.740909,7359.488462,Abita Brewing Co.,"Abita Springs, LA",43.0,78.0,3.44,880.0,90.0,Vienna Lager,4.50%,http://beeradvocate.com/beer/profile/3/5
2214,Amber Road Amber Ale,23922.000000,1.322800e+09,3.675000,3.525000,3.950000,3.625000,3.550000,5.500000,74632.000000,Baxter Brewing Co.,"Lewiston, ME",114.0,82.0,3.61,86.0,87.0,American Amber / Red Ale,5.50%,http://beeradvocate.com/beer/profile/23922/74632
2788,Apollo,10607.000000,1.216061e+09,4.089744,3.807692,3.923077,3.961538,3.961538,5.500000,31247.000000,Sixpoint Brewery,"Brooklyn, NY",4.0,88.0,3.94,421.0,95.0,Kristalweizen,5.20%,http://beeradvocate.com/beer/profile/10607/80745
3737,Back Bay IPA,9343.000000,1.167048e+09,3.892857,3.871429,3.828571,3.807143,3.871429,6.500000,552.000000,Beer Works,United States,62.0,86.0,3.84,103.0,98.0,American IPA,6.80%,http://beeradvocate.com/beer/profile/9343/552
3872,Baggywrinkle Barleywine,261.000000,1.179077e+09,3.285714,3.600000,3.600000,3.300000,3.428571,12.000000,2237.000000,Cisco Brewers Inc.,"Nantucket, MA",21.0,80.0,3.49,106.0,83.0,English Barleywine,12.00%,http://beeradvocate.com/beer/profile/261/2237
4873,Be Hoppy,22200.000000,1.300908e+09,4.166667,4.150000,4.183333,4.050000,4.116667,6.500000,57223.000000,Wormtown Brewing Co.,"Worcester, MA",96.0,94.0,4.27,175.0,100.0,American IPA,6.50%,http://beeradvocate.com/beer/profile/22200/57223
5312,Belgian White,1085.829016,1.237963e+09,3.606218,3.528497,3.512953,3.448187,3.466321,4.725882,38402.388601,Long Trail Brewing Co.,"Bridgewaters Corners, VT",111.0,79.0,3.47,284.0,74.0,Witbier,4.70%,http://beeradvocate.com/beer/profile/94/41182
